In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    # Используем параметр p
    recs = np.random.choice(items,
                            size=n,
                            replace=False,
                            p=weights)

    
    return recs.tolist()

In [5]:
%%time

# your_code
# Считаем продажи каждого товара, берем логарифм
# Полученные значения делаем положительными и приводим к единичной сумме
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['sales_value'] = np.log2(item_weights['sales_value'] + 0.001)
item_weights['sales_value'] = item_weights['sales_value'] - item_weights['sales_value'].min()
item_weights['sales_value'] = item_weights['sales_value'] / item_weights['sales_value'].sum()
item_weights = item_weights.rename(columns={'sales_value': 'weight'})
item_weights

Wall time: 100 ms


,item_id,weight
0,25671,0.000012
1,26081,0.000008
2,26093,0.000009
3,26190,0.000009
4,26355,0.000009
...,...,...
86860,17381856,0.000000
86861,17382205,0.000011
86862,17383227,0.000010
86863,17827644,0.000010


In [9]:
%%time

result['w_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))
result.head(2)

Wall time: 2.85 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,w_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[10204780, 2664434, 819690, 58868, 13007018]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[1010011, 1073086, 10149489, 977332, 7462511]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [10]:
result = pd.read_csv('data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [15]:
from imetrics import precision_at_k

In [17]:
# # your_code
# rec_metrics = {}
# for col in result.columns[1:]:
#     rec_metrics[col] = round(result.apply(lambda row: precision_at_k(row[col], row['actual']), axis=1).mean(), 4)

In [18]:
def eval_actual(item):
    garbage = ['[', ']', '\n']
    for g in garbage:
        item = item.replace(g, '')
    item = list(map(int, item.split()))
    return item

def precision_at_k(x, k=5):
    recommended_list = x[0]
    bought_list = x[1]
    
    bought_list = eval_actual(bought_list)
    bought_list = np.array(bought_list)
    
    recommended_list = eval(recommended_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

k = 5
model_names = result.columns.tolist()[2:]
for model in model_names:
    result[f'{model}_precision@{k}'] = result[[model, 'actual']].apply(precision_at_k, axis=1)

model_mask = ['@' in col for col in result.columns]
cols = result.columns.tolist()
model_cols = [cols[i] for i in range(len(cols)) if model_mask[i]]

result[model_cols].mean()

random_recommendation_precision@5     0.000588
popular_recommendation_precision@5    0.155240
itemitem_precision@5                  0.033595
cosine_precision@5                    0.035260
tfidf_precision@5                     0.036141
own_purchases_precision@5             0.179987
dtype: float64

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code